In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import stan
import arviz as az
import nest_asyncio
nest_asyncio.apply()
import pandas as pd

In [3]:
df = pd.read_csv('insurance.csv', delimiter=',')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
half_len = int(df.charges.shape[0]/2)
df_split = [df[i:i+half_len] for i in range(0,len(df), half_len)]

dftrain = df_split[0]
dftest = df_split[1]

In [5]:
age_mean = np.mean(dftrain.age.values)
age_dev = np.std(dftrain.age.values)
age_standard = (dftrain.age.values - age_mean)/age_dev

bmi_mean = np.mean(dftrain.bmi.values)
bmi_dev = np.std(dftrain.bmi.values)
bmi_standard = (dftrain.bmi.values - bmi_mean)/bmi_dev

children_mean = np.mean(dftrain.children.values)
children_dev = np.std(dftrain.children.values)
children_standard = (dftrain.children.values - children_mean)/children_dev

X = np.column_stack((age_standard, bmi_standard, children_standard))

charges_mean = np.mean(dftrain.charges.values)
charges_dev = np.std(dftrain.charges.values)
charges_standard = (dftrain.charges.values - charges_mean)/charges_dev

data_dict = {
    'y': charges_standard, 
    'x': X,
    'K': 3,
    'N': dftrain.charges.shape[0]
}

In [11]:
program_code = """
data {
    int<lower=1> N;
    int<lower=1> K;
    matrix[N, K] x;
    vector[N] y;
}

parameters {
    real alpha;
    vector[K] beta;
    real<lower=0> sigma;
}

model {
    sigma ~ inv_gamma(2, 3);
    alpha ~ normal(0, 10);
    beta ~ multi_normal(rep_vector(0, K), 10 * diag_matrix(rep_vector(1, K)));
    y ~ normal(x * beta + alpha, sigma);
}

generated quantities{
    vector[N] y_pred;
    for (i in 1:N) {
        y_pred[i] = normal_rng(x[i] * beta + alpha, sigma);
    }
}
"""

In [12]:
model = stan.build(program_code, data=data_dict)

Building...



Building: 23.6s, done.

In [13]:
fit = model.sample(num_chains=4, num_warmup=1000, num_samples=2500)

Sampling:   0%
Sampling:  25% (3500/14000)
Sampling:  50% (7000/14000)
Sampling:  75% (10500/14000)
Sampling: 100% (14000/14000)
Sampling: 100% (14000/14000), done.
Messages received during sampling:
  Gradient evaluation took 0.000118 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.18 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.65 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.55 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.58 seconds.
  Adjust your expectations accordingly!


In [14]:
df2 = fit.to_frame()
df2.head()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,beta.1,beta.2,...,y_pred.660,y_pred.661,y_pred.662,y_pred.663,y_pred.664,y_pred.665,y_pred.666,y_pred.667,y_pred.668,y_pred.669
draws,,,,,,,,,,,,,,,,,,,,,
0,-293.686859,0.997929,0.698099,3.0,7.0,0.0,294.783953,-0.015405,0.303986,0.134341,...,1.328596,-0.006410,-1.170316,-0.135054,-0.033524,1.740365,2.006802,0.578179,0.331257,0.915817
1,-293.273862,0.918065,0.698731,3.0,7.0,0.0,296.654485,-0.020283,0.319539,0.104377,...,0.376469,-0.535850,0.673152,0.981346,-0.095067,-0.417018,1.367020,-0.021906,-0.931701,0.811786
2,-291.764561,1.000000,0.790634,2.0,3.0,0.0,296.040622,0.003174,0.318199,0.155755,...,1.933544,1.443413,0.572839,0.645941,-2.542058,-0.285255,0.269700,1.897424,0.483446,-0.669782
3,-293.373793,0.974197,0.792345,2.0,3.0,0.0,297.100554,-0.027514,0.294716,0.114007,...,0.335460,0.524581,0.784686,-1.101356,-0.542487,-0.774308,0.845700,0.807339,-1.540116,0.818132
4,-295.157023,0.925918,0.698099,3.0,7.0,0.0,296.262002,0.013764,0.293560,0.121498,...,0.253973,0.230606,0.334420,0.903797,-0.178326,0.013010,-0.100373,1.034752,-0.542711,1.071458


In [15]:
az.summary(fit)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.000,0.036,-0.073,0.065,0.000,0.000,12479.0,8045.0,1.0
beta[0],0.308,0.037,0.239,0.378,0.000,0.000,11632.0,7850.0,1.0
beta[1],0.146,0.037,0.078,0.215,0.000,0.000,12629.0,7862.0,1.0
beta[2],0.022,0.037,-0.047,0.089,0.000,0.000,12907.0,7801.0,1.0
sigma,0.938,0.026,0.889,0.986,0.000,0.000,12562.0,7838.0,1.0
...,...,...,...,...,...,...,...,...,...
y_pred[664],0.299,0.944,-1.490,2.055,0.010,0.007,9358.0,10058.0,1.0
y_pred[665],0.266,0.942,-1.496,2.039,0.010,0.007,9126.0,9541.0,1.0
y_pred[666],0.144,0.931,-1.539,1.976,0.009,0.007,9962.0,9929.0,1.0
y_pred[667],0.059,0.946,-1.719,1.877,0.010,0.007,9643.0,9467.0,1.0


PROBLEM 5 ANSWER

First, to check the convergence and efficiency of the results, the summary shows the r_hat of the data to be less than 1.05, and the sampling efficiency of ess to be greater than 100, which should support the results' validity. Looking at the 3 variables, it seems beta[0] has the highest mean value at 0.3. This indicates that beta[0], which represents age, should be the strongest predictor for insurance charges out of the 3 variables tested.

In [16]:
age_mean = np.mean(dftest.age.values)
age_dev = np.std(dftest.age.values)
age_standard = (dftest.age.values - age_mean)/age_dev

bmi_mean = np.mean(dftest.bmi.values)
bmi_dev = np.std(dftest.bmi.values)
bmi_standard = (dftest.bmi.values - bmi_mean)/bmi_dev

children_mean = np.mean(dftest.children.values)
children_dev = np.std(dftest.children.values)
children_standard = (dftest.children.values - children_mean)/children_dev

X = np.column_stack((age_standard, bmi_standard, children_standard))

charges_mean = np.mean(dftest.charges.values)
charges_dev = np.std(dftest.charges.values)
charges_standard = (dftest.charges.values - charges_mean)/charges_dev

data_dict_test = {
    'y': charges_standard, 
    'x': X,
    'K': 3,
    'N': dftest.charges.shape[0]
}

In [17]:
model = stan.build(program_code, data=data_dict_test)

Building...



Building: found in cache, done.

In [18]:
fit = model.sample(num_chains=4, num_warmup=1000, num_samples=2500)

Sampling:   0%
Sampling:  25% (3500/14000)
Sampling:  50% (7000/14000)
Sampling:  75% (10500/14000)
Sampling: 100% (14000/14000)
Sampling: 100% (14000/14000), done.
Messages received during sampling:
  Gradient evaluation took 6.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.67 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.92 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.61 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.48 seconds.
  Adjust your expectations accordingly!


In [19]:
df2 = fit.to_frame()
df2.head()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,alpha,beta.1,beta.2,...,y_pred.660,y_pred.661,y_pred.662,y_pred.663,y_pred.664,y_pred.665,y_pred.666,y_pred.667,y_pred.668,y_pred.669
draws,,,,,,,,,,,,,,,,,,,,,
0,-297.659569,0.887260,0.776943,2.0,3.0,0.0,302.058619,0.020016,0.275592,0.187877,...,1.776669,-0.130831,0.532734,-0.574734,-0.161748,-0.715598,-0.713093,0.603018,-0.982641,-1.079816
1,-299.082345,0.813217,0.784533,2.0,7.0,0.0,301.007403,0.014128,0.241493,0.131242,...,0.418014,0.781088,0.551888,0.988432,1.729688,-0.315688,-2.392171,-0.866956,-0.585894,2.190305
2,-298.975632,0.858596,0.791402,2.0,3.0,0.0,299.370287,0.052723,0.274509,0.195596,...,-1.368145,1.196880,0.854316,-1.484645,0.963479,-0.597652,-0.566272,0.204133,1.411183,-0.900559
3,-298.854284,0.920089,0.830941,3.0,7.0,0.0,301.248218,0.021453,0.224532,0.198264,...,-0.399242,0.852052,0.901631,-1.248571,0.749405,-0.781217,0.343981,0.553885,-2.243047,0.699014
4,-300.877501,0.762748,0.776943,2.0,3.0,0.0,301.599566,-0.001548,0.232958,0.123707,...,0.171625,-0.112982,1.643623,1.307624,0.219211,1.434134,-0.482616,-0.244177,-0.413229,0.444399


In [30]:
df3 = az.summary(fit)
df3

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.000,0.037,-0.068,0.070,0.000,0.000,12704.0,7599.0,1.0
beta[0],0.250,0.037,0.180,0.319,0.000,0.000,12746.0,8248.0,1.0
beta[1],0.189,0.036,0.121,0.258,0.000,0.000,11543.0,7651.0,1.0
beta[2],0.086,0.037,0.018,0.156,0.000,0.000,11268.0,7823.0,1.0
sigma,0.946,0.026,0.895,0.993,0.000,0.000,12272.0,7777.0,1.0
...,...,...,...,...,...,...,...,...,...
y_pred[664],0.359,0.953,-1.397,2.174,0.010,0.007,9963.0,9891.0,1.0
y_pred[665],-0.412,0.948,-2.119,1.444,0.009,0.007,9971.0,9649.0,1.0
y_pred[666],-0.274,0.946,-2.068,1.463,0.009,0.007,9986.0,9853.0,1.0
y_pred[667],-0.552,0.940,-2.316,1.193,0.010,0.007,9775.0,8790.0,1.0


In [46]:
summation = 0

for i in range(6, 674):
    pred_mean = df3.iloc[i, 0]
    summation+=np.square(pred_mean - charges_standard[i-6])

temp = summation/dftest.charges.shape[0]

RMSE = np.sqrt(temp)
RMSE

1.0702449986219869

PROBLEM 6 ANSWER

The RMSE between the actual test value and the predicted means is higher than what would be considered good accuracy. This shows that the model's predictions are not as accurate as it could be, but it also shows that they are not extremely far off either. 
By computing the predictive means, the individual difference values are lost for each test instance. 
The uncertainly information in the predictive distribution can be found by sampling from that distribution, then calculating the RMSEs for those sample collections. The RMSEs then form a distribution.